# Система Программирования Квантового Сопроцессора 
(проект Python)

In [2]:
import numpy as np

## Класс QuantumCircuit
Класс, предоставляющий интерфейс описания и задания структуры квантовой схемы

*Свойства*:
* num_logical_qubits
* operations
* gate_symbols

*Методы*:
* rx
* rz
* cnot
* draw

In [3]:
class QuantumCircuit:
    def __init__(self, num_logical_qubits):
        self.num_logical_qubits = num_logical_qubits
        self.operations = []
        self.gate_symbols = {
            'rx': 'Rx',
            'rz': 'Rz',
            'cnot': 'X'
        }
    
    def rx(self, theta, qubit):
        self.operations.append({'name': 'rx', 'theta': theta, 'qubit': qubit})
    
    def rz(self, theta, qubit):
        self.operations.append({'name': 'rz', 'theta': theta, 'qubit': qubit})
    
    def cnot(self, control, target):
        self.operations.append({'name': 'cnot', 'control': control, 'target': target})
    
    def draw(self):
        # Собираем все операции по слоям
        num_qubits = self.num_logical_qubits
        
        # Определяем ширину каждого столбца
        col_widths = []
        for op in self.operations:
            if op['name'] == 'cnot':
                col_widths.append(3)
            else:
                theta_str = f"({round(op['theta']/np.pi, 2)}π)" if op['theta'] else ""
                col_widths.append(len(self.gate_symbols[op['name']]) + len(theta_str) + 2)

        # Создаем матрицу для отображения
        grid = []
        connections = []
        for q in range(num_qubits):
            row = []
            for i, op in enumerate(self.operations):
                if op['name'] == 'cnot':
                    control = op['control']
                    target = op['target']
                    if q == control:
                        row.append('■' + '─'*(col_widths[i]-1))
                    elif q == target:
                        row.append('X' + '─'*(col_widths[i]-1))
                    else:
                        if min(control, target) < q < max(control, target):
                            row.append('│' + ' '*(col_widths[i]-1))
                            connections.append((i, q))
                        else:
                            row.append(' ' + '─'*(col_widths[i]-1))
                else:
                    if q == op['qubit']:
                        gate = self.gate_symbols[op['name']]
                        theta_str = f"({round(op['theta']/np.pi, 2)}π)" if op.get('theta') else ""
                        cell = f"{gate}{theta_str}"
                        row.append(cell.center(col_widths[i], '─'))
                    else:
                        row.append('─'*col_widths[i])
            grid.append(row)

        # Строим линии соединений
        connection_lines = {}
        for col, qubit in connections:
            if col not in connection_lines:
                connection_lines[col] = []
            connection_lines[col].append(qubit)

        # Формируем окончательный вывод
        output = []
        for q in range(num_qubits):
            line = [f"q{q}: ─"]
            for col, op in enumerate(self.operations):
                line.append(grid[q][col])
            output.append(''.join(line).rstrip('─') + '─')

        # Добавляем вертикальные связи
        for col in sorted(connection_lines.keys()):
            conn_line = [' '*(len(f"q{q}: ─")-1) for q in range(num_qubits)]
            for q in connection_lines[col]:
                conn_line[q] = ' '*(len(f"q{q}: ─")-1) + '│'
            output.insert(2*col+1, '  '.join(conn_line))

        return '\n'.join(output)

## Класс QuantumSimulator
Класс, предоставляющий симуляцию выполнения инструкций квантовой схемы. Переводит инструкции из логической абстракции в физическую, используя операции, определенные в архитектуре.

*Свойства*:
* num_physical_qubits
* state

*Методы*:
* apply_iswap
* apply_phase
* apply_cswap
* execute
* get_logical_state
* get_probabilities

In [4]:

class QuantumSimulator:
    def __init__(self, num_logical_qubits):
        self.num_physical_qubits = 2 * num_logical_qubits
        self.state = np.zeros(2**self.num_physical_qubits, dtype=complex)
        # Инициализация в логический 0 (физические 01)
        index = 0
        for i in range(num_logical_qubits):
            phy1 = 2*i + 1
            index += 2**phy1
        self.state[index] = 1 + 0j

    def apply_iswap(self, q1, q2, theta):
        c = np.cos(theta)
        s = -1j * np.sin(theta)
        iswap = np.array([
            [1, 0, 0, 0],
            [0, c, s, 0],
            [0, s, c, 0],
            [0, 0, 0, 1]
        ], dtype=complex)
        new_state = np.zeros_like(self.state)
        for i in range(len(self.state)):
            bit_q1 = (i >> q1) & 1
            bit_q2 = (i >> q2) & 1
            idx = (bit_q1 << 1) | bit_q2
            rest = i & ~((1 << q1) | (1 << q2))
            for j in range(4):
                new_bit_q1 = (j >> 1) & 1
                new_bit_q2 = j & 1
                new_i = rest | (new_bit_q1 << q1) | (new_bit_q2 << q2)
                new_state[new_i] += self.state[i] * iswap[idx, j]
        self.state = new_state
    
    def apply_phase(self, q1,q2, theta, phi=0):
        
        # сделать phi = 0, чтоьы получилась 1
        # phase_factors = np.array([
        #     np.exp(-1j * phi/2),    # |00⟩
        #     np.exp(-1j * theta/2),  # |01⟩
        #     np.exp(1j * theta/2),   # |10⟩
        #     np.exp(1j * phi/2)      # |11⟩
        # ], dtype=np.complex128)
         
        phase = np.exp(1j * theta/2)
        phase2 = np.exp(-1j * theta/2)
        for i in range(len(self.state)):
            if (i >> q1) & 1:
                self.state[i] *= phase
            if (i >> q2) & 1:
                self.state[i] *= phase2
        # print(self.state)
    
    def apply_cswap(self, control, target1, target2):
        new_state = np.zeros_like(self.state)
        for i in range(len(self.state)):
            c_bit = (i >> control) & 1
            t1 = (i >> target1) & 1
            t2 = (i >> target2) & 1
            if c_bit:
                new_t1, new_t2 = t2, t1
            else:
                new_t1, new_t2 = t1, t2
            new_i = i & ~((1 << target1) | (1 << target2))
            new_i |= (new_t1 << target1) | (new_t2 << target2)
            new_state[new_i] = self.state[i]
        self.state = new_state
    
    def execute(self, circuit):
        for op in circuit.operations:
            if op['name'] == 'rx':
                logical_qubit = op['qubit']
                phy0 = 2 * logical_qubit
                phy1 = 2 * logical_qubit + 1
                self.apply_iswap(phy0, phy1, op['theta'] / 2)
            elif op['name'] == 'rz':
                logical_qubit = op['qubit']
                self.apply_phase(2 * logical_qubit,2 * logical_qubit+1, op['theta'])
            elif op['name'] == 'cnot':
                control = op['control']
                target = op['target']
                self.apply_cswap(2*control, 2*target, 2*target+1)
    
    def get_logical_state(self):
        logical_state = {}
        num_logical = self.num_physical_qubits // 2
        for i in range(len(self.state)):
            if abs(self.state[i]) < 1e-10:
                continue
            bits = [(i >> q) & 1 for q in range(self.num_physical_qubits)]
            logical_bits = []
            valid = True
            for l in range(num_logical):
                phy0, phy1 = 2*l, 2*l + 1
                b0, b1 = bits[phy0], bits[phy1]
                if (b0, b1) == (0, 1):
                    logical_bits.append(0)
                elif (b0, b1) == (1, 0):
                    logical_bits.append(1)
                else:
                    valid = False
                    break
            if valid:
                logical_index = sum(bit << idx for idx, bit in enumerate(logical_bits))
                logical_state[logical_index] = logical_state.get(logical_index, 0) + self.state[i]
        # Normalize
        norm = np.sqrt(sum(abs(v)**2 for v in logical_state.values()))
        return {f"{bin(k)[2:].zfill(num_logical)}": v/norm for k, v in logical_state.items()}
    
    def get_probabilities(self, decimals=5):
        """Возвращает словарь с вероятностями логических состояний"""
        logical_state = self.get_logical_state()
        return {
            state: round(abs(amp)**2, decimals)
            for state, amp in logical_state.items()
            if round(abs(amp)**2, decimals) > 0
        }

In [63]:
# Пример использования
circuit = QuantumCircuit(1) # Инициализируем экземпляр схемы
circuit.rx(np.pi, 0) # Добавляем операторы

print("Схема:")
print(circuit.draw()) # Отрисовываем схему

sim = QuantumSimulator(1) # Инициализируем экземпляр симулятора
sim.execute(circuit) # запускаем работу симулятора с конкретной схемой

print(sim.get_logical_state()) # Выводим данные
print("\nВероятности:")
print(sim.get_probabilities())

Схема:
q0: ──Rx(1.0π)─
{'1': np.complex128(-1j)}

Вероятности:
{'1': np.float64(1.0)}


In [5]:
# Создаем цепь с 1 логическим кубитом
circuit = QuantumCircuit(1)
circuit.rz(np.pi, 0)  # R_x(π) переворачивает |0⟩ → |1⟩
print("Визуализация схемы:")
print(circuit.draw())

sim = QuantumSimulator(1)
sim.execute(circuit)
print(sim.get_logical_state())  # {1: (1+0j)}
print("\nВероятности состояний:")
print(sim.get_probabilities())

Визуализация схемы:
q0: ──Rz(1.0π)─
{'0': np.complex128(6.123233995736766e-17-1j)}

Вероятности состояний:
{'0': np.float64(1.0)}


In [15]:

# Создаем цепь с 2 логическими кубитами
circuit = QuantumCircuit(1)

# Применяем гейты
circuit.rx(np.pi, 0)  # Создание суперпозиции
# circuit.rz(np.pi, 0)  # Добавление фазы
# circuit.cnot(0, 1)      # Запутывание кубитов
print("Визуализация схемы:")
print(circuit.draw())

sim = QuantumSimulator(1)
sim.execute(circuit)
print("Состояние Белла:")
print(sim.get_logical_state())  # Пример: {0: 0.707, 3: 0.707j}
print("\nВероятности состояний:")
print(sim.get_probabilities())

Визуализация схемы:
q0: ──Rx(1.0π)─
Состояние Белла:
{'1': np.complex128(-1j)}

Вероятности состояний:
{'1': np.float64(1.0)}


In [249]:
circuit = QuantumCircuit(3)
circuit.rx(np.pi/2, 0)
circuit.cnot(0, 2)
circuit.rz(np.pi/4, 1)
circuit.cnot(1, 2)
print(circuit.draw())

sim = QuantumSimulator(3)
sim.execute(circuit)
print(sim.get_logical_state())  # Пример: {0: 0.707, 3: 0.707j}
print("\nВероятности состояний:")
print(sim.get_probabilities())

q0: ──Rx(0.5π)─■───────────── ─
q1: ───────────│  ─Rz(0.25π)─■─
q2: ───────────X─────────────X─
          │      
{'101': np.complex128(0.7071067811865475j), '000': np.complex128(0.7071067811865476+0j)}

Вероятности состояний:
{'101': np.float64(0.5), '000': np.float64(0.5)}
